<a href="https://colab.research.google.com/github/subhamkapoor360/MLHelpers/blob/master/Cactus_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# drive.mount('/content/drive/',force_remount=True)
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import shutil
import glob
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [3]:
!ls "/content/drive/My Drive/Datasets/cactus-identification/"

sample_submission.csv  test  test.zip  train  train.csv  train.zip


In [0]:
BASE_DIR = "/content/drive/My Drive/Datasets/cactus-identification/"

In [0]:
train = pd.read_csv("{}{}".format(BASE_DIR,'train.csv'))

In [6]:
train.head(2)

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1


In [0]:
!unzip '/content/drive/My Drive/Datasets/cactus-identification/train.zip'
!unzip '/content/drive/My Drive/Datasets/cactus-identification/test.zip'

In [8]:
!ls /content/

drive  sample_data  test  train


In [9]:
train.head(2)

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1


In [0]:
base_dir = '/content/'
if not os.path.exists(base_dir):
  os.mkdir(base_dir)


#[CREATE FOLDERS INSIDE THE BASE DIRECTORY]

# now we create 2 folders inside 'base_dir':

# train_dir
    # has_cactus
    # has_no_cactus

# val_dir
    # has_cactus
    # has_no_cactus



# create a path to 'base_dir' to which we will join the names of the new folders
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
if not os.path.exists(train_dir):
  os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
if not os.path.exists(val_dir):
  os.mkdir(val_dir)



# [CREATE FOLDERS INSIDE THE TRAIN AND VALIDATION FOLDERS]
# Inside each folder we create seperate folders for each class

# create new folders inside train_dir
has_cactus = os.path.join(train_dir, 'has_cactus')
if not os.path.exists(has_cactus):
  os.mkdir(has_cactus)
has_no_cactus = os.path.join(train_dir, 'has_no_cactus')
if not os.path.exists(has_no_cactus):
  os.mkdir(has_no_cactus)


# create new folders inside val_dir
has_cactus = os.path.join(val_dir, 'has_cactus')
if not os.path.exists(has_cactus):
  os.mkdir(has_cactus)
has_no_cactus = os.path.join(val_dir, 'has_no_cactus')
if not os.path.exists(has_no_cactus):
  os.mkdir(has_no_cactus)

In [0]:
train.index = train.id

In [12]:

df_train, df_val = train_test_split(train, test_size=0.20, random_state=101,)
# if K.image_data_format() == 'channels_first':
#     df_train = df_train.reshape(df_train.shape[0], IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
#     df_val = df_val.reshape(df_val.shape[0], 1, IMAGE_SIZE, IMAGE_SIZE)
#     input_shape = (1, img_rows, img_cols)
# else:
#     df_train = df_train.reshape(df_train.shape[0], IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
#     df_val = df_val.reshape(df_val.shape[0], IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
#     input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
print(df_train.shape)
print(df_val.shape)

(14000, 2)
(3500, 2)


In [13]:
if not os.path.exists('test'):
  os.mkdir('test')
if not os.path.exists('test/test'):
  os.mkdir('test/test')
!mv test/* test/test/

mv: cannot move 'test/test' to a subdirectory of itself, 'test/test/test'


In [0]:
train_list = list(df_train['id'])
val_list = list(df_val['id'])



# Transfer the train images

for image in train_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image
    # get the label for a certain image
    target = train.loc[image,'has_cactus']
    
    # these must match the folder names
    if target == 0:
        label = 'has_no_cactus'
    if target == 1:
        label = 'has_cactus'
    
    # source path to image
    src = os.path.join('train', fname)
    # destination path to image
    dst = os.path.join(train_dir, label, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)


# Transfer the val images

for image in val_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image
    # get the label for a certain image
    target = train.loc[image,'has_cactus']
    
    # these must match the folder names
    if target == 0:
        label = 'has_no_cactus'
    if target == 1:
        label = 'has_cactus'
    

    # source path to image
    src = os.path.join('train', fname)
    # destination path to image
    dst = os.path.join(val_dir, label, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)
    

In [0]:
IMAGE_SIZE = 32
IMAGE_CHANNELS = 3


In [0]:
train_path = 'train_dir'
valid_path = 'val_dir'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [17]:
datagen = ImageDataGenerator(rescale=1.0/255,
                             horizontal_flip = True,
                             vertical_flip = True,
                             featurewise_center = True,
                             samplewise_center = True,
                             featurewise_std_normalization = True,
                             samplewise_std_normalization = True
                            )

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

Found 14000 images belonging to 2 classes.
Found 3500 images belonging to 2 classes.


In [18]:
print(val_gen.class_indices)

{'has_cactus': 0, 'has_no_cactus': 1}


In [19]:
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)    

In [0]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', 
              metrics=['accuracy'])


In [21]:
if not os.path.exists('MLModels'):
  os.mkdir('MLModels')
filepath = "MLModels/Cactus-identification.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=15, verbose=1,
                   callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:707: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/15
350/350 [==============================] - 5s 13ms/step - loss: 0.1080 - acc: 0.9597

Epoch 00001: val_acc improved from -inf to 0.95971, saving model to MLModels/Cactus-identification.h5
1400/1400 [==============================] - 56s 40ms/step - loss: 0.1756 - acc: 0.9281 - val_loss: 0.1080 - val_acc: 0.9597
Epoch 2/15
350/350 [==============================] - 5s 14ms/step - loss: 0.0693 - acc: 0.9726

Epoch 00002: val_acc improved from 0.95971 to 0.97257, saving model to MLModels/Cactus-identification.h5
1400/1400 [==============================] - 47s 34ms/step - loss: 0.0949 - acc: 0.9639 - val_loss: 0.0693 - val_acc: 0.9726
Epoch 3/15
350/350 [==============================] - 4s 12ms/step - loss: 0.0626 - acc: 0.9771

Epoch 00003: val_acc improved from 0.97257 to 0.97714, saving model to MLModels/Cactus-identification.h5
1400/1400 [==============================] - 46s 33ms/step - loss: 0.0693 - acc: 0.9744 - val_loss: 0.0626 - val_acc: 0.9771
Epoch 4/15
350/350 [==

In [0]:
has_cactus_sample = img_to_array(load_img('train_dir/has_cactus/0148bb4a295cf49c0169d69a4a63df7e.jpg'))
has_no_cactus_sample = img_to_array(load_img('train_dir/has_no_cactus/04108a6d0299da92ebee86db8fc77bb0.jpg'))



In [23]:
model = load_model('MLModels/Cactus-identification.h5')

pred_1 = np.argmax(model.predict(has_cactus_sample.reshape(1,  IMAGE_SIZE, IMAGE_SIZE,IMAGE_CHANNELS,)))
pred_2 = np.argmax(model.predict(has_no_cactus_sample.reshape(1,  IMAGE_SIZE, IMAGE_SIZE,IMAGE_CHANNELS,)))
rev_dict = {v:k for k,v in val_gen.class_indices.items()}
print(pred_1,pred_2)
print(rev_dict[pred_1],rev_dict[pred_2])

0 1
has_cactus has_no_cactus


In [24]:
test_gen = datagen.flow_from_directory(directory=r"./test",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 4000 images belonging to 1 classes.


In [25]:
prediction = model.predict_generator(test_gen,steps = len(test_gen))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:707: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
